In [93]:
import advent
import numpy as np

data = advent.get_char_grid(10, 'txt')
# Pad the array so we don't have to keep checking if indexes are out of bounds
data = np.pad(data, 1, 'constant', constant_values='.')

start = np.where(data == 'S')
start = start[0][0], start[1][0]
start

(43, 26)

In [94]:
offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)]
tadd = lambda a, b: (a[0] + b[0], a[1] + b[1])

def step(data, pos, excluded_pos):
    # Return next position in the pipe. Never return excl_pos (so you don't go back in the pipe)
    for offset in offsets:
        new_pos = tadd(pos, offset)
        if new_pos == excluded_pos: continue
        o, c = data[pos], data[new_pos]
        if offset == (1, 0) and o in 'F7|S' and c in 'LJ|S': return new_pos
        if offset == (-1, 0) and o in 'LJ|S' and c in 'F7|S': return new_pos
        if offset == (0, 1) and o in 'FL-S' and c in '7J-S': return new_pos
        if offset == (0, -1) and o in '7J-S' and c in 'FL-S': return new_pos
    raise ValueError(f"This position does not connect in any direction: {pos}. ({excluded_pos})")

pipe_length = 0
position, previous = start, (0, 0)
path = []
while True:
    path += [position]
    pipe_length += 1
    position, previous = step(data, position, previous), position
    if data[position] == 'S': break

print(pipe_length // 2)


7173


In [95]:
# There is a clever way to calculate whether a tile is in the loop. Then we just brute force that method for each tile
# The method: start at the tile, start going up (or any direction, but lets just do up to keep it easy)
# If we cross an EVEN number of pipes, we are OUTSIDE. if we cross an ODD number of pipes, we are INSIDE

# Unfortunately, a cross is a bit tough, because both 'J -> 7' and 'L -> F' count as 2 crosses
# Whereas '-', or 'J -> F' or 'L -> '7' count as 1 cross
# I could have done this easier with regexes, but I only figured that out when I was nearly done with the for loop

# Final unfortunately: the 'S' does need to be filled in actually (yeah this exercise turned out to be harder than expected :( )

def fill_s(data, path):
    # SUPER HACKY !!!!
    # But this function would essentially just be 6 'if' statements (one each for -|7FJL)
    # so not very interesting, and it's easy to find the value of S by eye
    return '7'


def step_off(start, end):
    if end == '-': return 1
    if (start == 'J' and end == 'F') or (start == 'L' and end == '7'): return 1
    if (start == 'J' and end == '7') or (start == 'L' and end == 'F'): return 2
    raise ValueError(f"{start}, {end}, unknown")

def is_inside(data, path: list[tuple[int, int]], position: tuple[int, int]) -> bool:
    if position in path: return False # edge case
    crosses = 0
    on_pipe = None # Where we started our pipe
    while position[0] > 0:
        position = tadd(position, (-1, 0))
        if data[position] in '7F' and position in path: # stepping off the pipe
            crosses += step_off(on_pipe, data[position])
            on_pipe = None
        elif data[position] in 'JL' and position in path: # stepping on the pipe
            on_pipe = data[position]
        elif data[position] in '-' and position in path: # simple cross
            crosses += 1
    return crosses % 2 == 1

In [97]:
from tqdm.notebook import trange

data[path[0]] = fill_s(data, path)

insiders = 0
for j in trange(data.shape[1]):
    for i in range(data.shape[0]):
        #if is_inside(data, path, (i, j)): print(i, j, data[(i, j)])
        if is_inside(data, path, (i, j)): insiders +=1

print(insiders) # Took like 100 seconds

  0%|          | 0/142 [00:00<?, ?it/s]

291


In [ ]:
# Optional: A way to visualize the map without the unneccesary pipe pieces
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if (i, j) not in path:
            data[(i, j)] = '.'
[print(''.join(row)) for row in data]